In [1]:
# https://cran.r-project.org/web/packages/penaltyLearning/index.html

In [2]:
# libraries
library(penaltyLearning)
library(data.table)
library(future)
library(future.apply)
library(directlabels)

In [3]:
# choosing training data
dataset <- "epigenomic"

In [4]:
# read data
feature.dt <- fread(paste("../../training_data/", dataset, "/inputs_old.csv", sep = ""))
target.dt  <- fread(paste("../../training_data/", dataset, "/outputs.csv", sep = ""))
folds.dt   <- fread(paste("../../training_data/", dataset, "/folds.csv", sep = ""))

In [5]:
# filter censored intervals
target.dt  <- target.dt[!(min.log.lambda == -Inf & max.log.lambda == Inf)]
feature.dt <- feature.dt[sequenceID %in% target.dt$sequenceID]
folds.dt   <- folds.dt[sequenceID %in% target.dt$sequenceID]

In [6]:
# combine
feature.dt.all <- cbind(feature.dt, folds.dt[, "fold"])
target.dt.all  <- cbind(target.dt, folds.dt[, "fold"])

In [7]:
# get number of folds
n_folds = dim(unique(folds.dt[,"fold"]))[1]
n_folds

[1] 6

In [8]:
for (test.fold in 1:n_folds) {
    # get train and test data
    feature.mat.train <- as.matrix(feature.dt.all[feature.dt.all$fold != test.fold][, 2:118])
    feature.mat.test  <- as.matrix(feature.dt.all[feature.dt.all$fold == test.fold][, 2:118])
    target.mat.train  <- as.matrix(target.dt.all[target.dt.all$fold != test.fold, c("min.log.lambda", "max.log.lambda")])
    
    # train model
    fit <- IntervalRegressionCV(feature.mat = feature.mat.train, target.mat = target.mat.train)
    
    # get prediction from test set
    target.mat.pred <- fit$predict(feature.mat.test)
    colnames(target.mat.pred) <- "llda"

    # save prediction to csv
    prediction <- data.frame(as.matrix(feature.dt.all[feature.dt.all$fold == test.fold, "sequenceID"]), target.mat.pred)
    write.csv(prediction, file = paste("predictions/", dataset, test.fold, "117", "csv", sep = "."), row.names = FALSE)
}